---
# Demo Use Case - Queries and Analytics on Video (Part 3)

The data scientist now wants to focus on video frames in which people were detected.

The metadata for the video stream produced by the object detection job is loaded into a Pandas dataframe.
This dataframe is used to allow the data scientist to view any image stored in SDP.
They can also filter, sort, and aggregate the dataframe using methods that data scientists are familar with.

---

### Install dependencies

See [install_dependencies.ipynb](install_dependencies.ipynb).

### How to use this Notebook
1. Click *Kernel* -> *Restart Kernel and Run All Cells*.

### Import dependencies

In [1]:
%load_ext autoreload
%autoreload 2

from matplotlib import pyplot as plt
import IPython
import cv2
import itertools
import numpy as np
import pandas as pd
import json
import base64
import datetime
import time
from ipywidgets import interact, interactive, fixed, interact_manual
from IPython.display import display
import ipywidgets as widgets
from pathlib import Path
import grpc
import imp
import pravega.grpc_gateway as pravega
import pravega.video as video
from pravega.video import UnindexedStream, OutputStream, IndexedStream, opencv_image_to_mpl, VideoPlayer
from matplotlib import pyplot as plt
from copy import copy
import os

imp.reload(video);

### Define Pravega stream parameters

In [2]:
#gateway = os.environ['PRAVEGA_GRPC_GATEWAY_ADDRESS']
#gateway = '10.246.27.131:54672'
gateway = 'pravega-grpc-gateway.examples.frightful-four.eaglemonk.intranet.nautilus-platform-dev.com:80'
scope = 'examples'
#stream = 'object-detector-input-video'
#stream = 'object-detector-output-video'
stream = 'virat-1'

### Initialize connection to Pravega GRPC Gateway

In [3]:
pravega_channel = grpc.insecure_channel(gateway, options=[
        ('grpc.max_receive_message_length', 9*1024*1024),
    ])
pravega_client = pravega.grpc.PravegaGatewayStub(pravega_channel)

### Load timestamp index
This is an index from timestamp to begin stream cut, end stream cut, and event pointer.
This index is maintained by the job in [index_builder.ipynb](index_builder.ipynb).
The video player uses the event pointer to fetch an individual frame. It uses the end stream cut to play from a particular frame.

In [4]:
indexed_stream = IndexedStream(pravega_client, scope, stream)

In [5]:
stream_info = indexed_stream.index_stream.get_stream_info()
index_size_MB = (list(stream_info.tail_stream_cut.cut.values())[0] - list(stream_info.head_stream_cut.cut.values())[0]) * 1e-6
index_size_MB

0.424325

stream_info = indexed_stream.get_stream_info()
stream_size_MB = (list(stream_info.tail_stream_cut.cut.values())[0] - list(stream_info.head_stream_cut.cut.values())[0]) * 1e-6
stream_size_MB

In [6]:
%%time
indexed_stream.load_index()

load_index: Reading index stream virat-1-index
CPU times: user 270 ms, sys: 86.8 ms, total: 357 ms
Wall time: 2.2 s


#### Show number of records in the index, first and last index records

In [7]:
len(indexed_stream.index_df)

1249

In [8]:
indexed_stream.index_df.iloc[[0,-1]].T

timestamp,2020-04-08 23:49:06.738000+00:00,2020-04-08 23:49:17.138000+00:00
camera,0,0
frameNumber,0,312
ssrc,0,0
to_stream_cut,H4sIAAAAAAAAADOwSq1IzC3ISS3WL8ssSizRNbQyAEJDQy...,H4sIAAAAAAAAADOwSq1IzC3ISS3WL8ssSizRNbQyAEJLcw...
event_pointer,AAABAAAAACIAG2V4YW1wbGVzL3ZpcmF0LTEvMC4jZXBvY2...,AAABAAAAACUAG2V4YW1wbGVzL3ZpcmF0LTEvMC4jZXBvY2...
from_stream_cut,H4sIAAAAAAAAADOwSq1IzC3ISS3WL8ssSizRNbQyAEEA55...,H4sIAAAAAAAAADOwSq1IzC3ISS3WL8ssSizRNbQyAEJLM0...


#### Clean recognitions

In [9]:
# def clean_recognitions(recognitions):
#     return ','.join(np.unique([r['title'] for r in recognitions]))
#indexed_stream.index_df['recog'] = indexed_stream.index_df.recognitions.apply(clean_recognitions)

### Video player with entire stream

In [16]:
player = VideoPlayer(stream=indexed_stream)
player.interact()

### Filtered and sorted player

In [11]:
indexed_stream.index_df.camera.value_counts()

0    313
3    312
2    312
1    312
Name: camera, dtype: int64

In [12]:
#indexed_stream.index_df.recog.value_counts()

In [13]:
#pd.DataFrame(indexed_stream.index_df.groupby(['camera','recog']).size()).unstack().fillna('-')

In [14]:
df = indexed_stream.index_df.copy()
#df = df[df.camera==0]
df = df[df.frameNumber.between(10,20)]
#df = df[df.recog!='']
#df = df[df.recog.str.contains('person')]
#df = df[df.recog.str.contains('boat')]
#df = df[df.recog.str.contains('bus,person')]
#df = df[df.recog.str.contains('motorbike')]
#df = df[df.recog.str.contains('train')]
#df = df[df.recog.str.contains('chair')]
#df = df[df.recog.str.contains('dog')]
#df = df[df.recog.str.contains('sofa')]
len(df)

44

In [15]:
filtered_stream = copy(indexed_stream)
filtered_stream.index_df = df
player = VideoPlayer(stream=filtered_stream)
player.interact()